# Import Libraries

In [52]:
import os
import PyPDF2
import openai
import uuid

import warnings
warnings.filterwarnings('ignore')

import pinecone
import tiktoken

from itertools import chain
from pinecone import Pinecone
from pinecone import ServerlessSpec
from Credentials import OPENAI_API_KEY, PINECONE_API_KEY

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone as lang_pinecone

# Load Data

In [2]:
os.chdir('..')
curr_dir = (os.getcwd()).replace('\\', '/')
file_path = curr_dir + '/Dataset/Corrective RAG.pdf'

In [22]:
tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [32]:
token_counts = tiktoken_len(text[0].page_content)
token_counts

791

In [44]:
def read_pdf(document):
       
    loader = PyPDFLoader(file_path)
    pdf_text = loader.load()
    pdf_text[0].page_content = pdf_text[0].page_content.replace('\n', '')

    return pdf_text    
        
def pdf_to_chunks(text, 
                  chunk_size, 
                  overlap):
    
    split_text = RecursiveCharacterTextSplitter(chunk_size=chunk_size, 
                                                chunk_overlap=overlap, 
                                                length_function=tiktoken_len)
    docs = split_text.split_text(text[0].page_content)
    
    chunks = [str(doc) for doc in docs]
        
    return chunks

In [39]:
text = read_pdf(file_path)
text[0].page_content

'Introduction to RAG  Retrieval -augmented generation (RAG) is an AI framework for improving the quality of  LLM -generated  responses by grounding the model on external sources of knowledge to supplement the LLM’s internal representation of information. Implementing RAG in an LLM -based question answering system has two main benefits: It ensures that the model has access to the  most current, reliable facts, and that users have access to the model’s sources, ensuring that its claims can be checked for accuracy and ultimately trusted.  Prior research has introduced retrieval techniques to incorporate relevant knowledge and augment  generation, as  exemplified  by retrieval  augmented  generation ( RAG). In this framework, the input to  models is augmented  by prepending  relevant  documents  that are retrieved  from  an external  knowledge  corpus .  While  RAG  serves  as a practicable complement to  LLMs, its effectiveness is contingent upon the relevance  and accuracy  of the retrie

In [49]:
pdf_chunks = pdf_to_chunks(text, 220, 10)
pdf_chunks

['Introduction to RAG  Retrieval -augmented generation (RAG) is an AI framework for improving the quality of  LLM -generated  responses by grounding the model on external sources of knowledge to supplement the LLM’s internal representation of information. Implementing RAG in an LLM -based question answering system has two main benefits: It ensures that the model has access to the  most current, reliable facts, and that users have access to the model’s sources, ensuring that its claims can be checked for accuracy and ultimately trusted.  Prior research has introduced retrieval techniques to incorporate relevant knowledge and augment  generation, as  exemplified  by retrieval  augmented  generation ( RAG). In this framework, the input to  models is augmented  by prepending  relevant  documents  that are retrieved  from  an external  knowledge  corpus .  While  RAG  serves  as a practicable complement to  LLMs, its effectiveness is contingent upon the relevance  and accuracy  of the retri

In [57]:
uid = str(uuid.uuid4())
uid

data = [
    {
        'id' : f'{uid}-{chunk_idx}',
        'text' : chunk
    } for chunk_idx, chunk in enumerate(pdf_chunks)
]
data

[{'id': 'e7842eaf-e492-4026-95cf-0c98b2ed5ccf-0',
  'text': 'Introduction to RAG  Retrieval -augmented generation (RAG) is an AI framework for improving the quality of  LLM -generated  responses by grounding the model on external sources of knowledge to supplement the LLM’s internal representation of information. Implementing RAG in an LLM -based question answering system has two main benefits: It ensures that the model has access to the  most current, reliable facts, and that users have access to the model’s sources, ensuring that its claims can be checked for accuracy and ultimately trusted.  Prior research has introduced retrieval techniques to incorporate relevant knowledge and augment  generation, as  exemplified  by retrieval  augmented  generation ( RAG). In this framework, the input to  models is augmented  by prepending  relevant  documents  that are retrieved  from  an external  knowledge  corpus .  While  RAG  serves  as a practicable complement to  LLMs, its effectiveness i

# Text Embedding and Storing Vector in Pinecone

In [9]:
pc = Pinecone(api_key=PINECONE_API_KEY)


    
index_name = 'chatbot'
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
                    dimension=1536,
                    metric='cosine',
                    spec=ServerlessSpec(
                        cloud='aws', 
                        region='us-west-2'
                    )
    )

index = pc.Index(index_name)
#openai.api_key = OPENAI_API_KEY
#model = 'text-embedding-ada-002'

# def create_embeddings(pdf_chunks):
    
#     embeddings_list = []
#     for chunk in pdf_chunks:
#         res = openai.Embedding.create(input=[chunk], engine=model)
#         embeddings_list.append(res['data'][0]['embedding'])
    
#     return embeddings_list

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
docsearch = Pinecone.from_documents(pdf_chunks, embeddings, index_name)


# pc = Pinecone(api_key=PINECONE_API_KEY)

# if index_name not in pc.list_indexes():
    
#     pc.create_index(index_name=index_name, 
#                     dimension=1536, 
#                     metric='cosine', 
#                     spec=ServerlessSpec(
#                         cloud='aws', 
#                         region='us-west-2'
#                     )
#     )

# index = pc.Index(index_name)

AttributeError: type object 'Pinecone' has no attribute 'from_documents'

In [23]:
embddings = create_embeddings(pdf_chunks, model)
embddings

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [9]:
vectors = embedding_vectorstore(pdf_chunks, PINECONE_API_KEY, OPENAI_API_KEY)
vectors

AttributeError: type object 'Pinecone' has no attribute 'from_documents'

# Store embeddings in Pinecone

In [9]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'chatbot'
# pc.create_index(index_name, 
#                 dimension=768,
#                 metric='cosine',
#                 spec=ServerlessSpec(
#                     cloud="aws",
#                     region="us-west-2"
#                 )
# )

index = pc.Index(index_name)

for idx, embedding in enumerate(embeddings):
    embedding_list = embedding.tolist()
    
    chunk_id = f'chunk_{idx+1}'
    index.upsert([embedding_list], [chunk_id])

ValueError: Invalid vector value passed: cannot interpret type <class 'list'>

In [15]:
len(embeddings)

9

In [11]:
pc.list_indexes()

{'indexes': [{'dimension': 768,
              'host': 'chatbot-wlh2nlv.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [33]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}